# 1.匯入 Dataset (panda對文字資料好用)

In [8]:
import pandas as pd
filepath='C:\\Users\\zxc98\\WordCloud/spam_1.csv'
def readData_rawSMS(filepath):
    #只需要columns 0，1
    data_rawSMS   = pd.read_csv(filepath,usecols=[0,1],encoding='latin-1')
    data_rawSMS.columns=['label','content']
    return   data_rawSMS
data_rawSMS = readData_rawSMS(filepath)

In [95]:
data_rawSMS.iloc[8].content
#data_rawSMS.head()


'WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# 2.這邊切割數據集單純用亂數取(0~1之間數字)
## >0.5   Training Data
## <=0.5 Test Data

In [49]:
import numpy as np
np.random.seed(5)
def Separate_TrainAndTest(data_rawSMS):
    n=int(data_rawSMS.shape[0])  #shape輸出為(行，列)
    tmp_train=(np.random.rand(n)>0.5)
    return data_rawSMS.iloc[np.where(tmp_train==True)[0]], data_rawSMS.iloc[np.where(tmp_train==False)[0]]
data_rawtrain,data_rawtest=Separate_TrainAndTest(data_rawSMS)
data_rawtest.head()

,label,content
0,ham,"Go until jurong point, crazy.. Available only ..."
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


# 3. 計算HAM和SPAM 的 TF-IDF差異 DIFF
## 值越大的代表他在SPAM的可能性更大
「size_table」: 要選多少個重要的「詞」出來，等於決定特徵向量的維度數。Default:我設成200。
 「ignore」: 英文字，字少於幾個以下就不要算，比如: 「I」就是1個字，「no」是2個字。Default:我設成3個。

In [87]:
import re
def generate_key_list(data_rawtrain, size_table=200,ignore=3):
    dict_spam_raw = dict()
    dict_genuine_raw = dict()
    dict_IDF = dict()

    # 去除字母外的所有內容.
    for i in range(data_rawSMS.shape[0]):
        finds = re.findall('[A-Za-z]+', data_rawSMS.iloc[i].content)
        if data_rawSMS.iloc[i].label == 'spam':
            for find in finds:
                if len(find)<ignore: continue
                find = find.lower() #英文轉成小寫
                try:
                    dict_spam_raw[find] = dict_spam_raw[find] + 1
                except:	
                    dict_spam_raw[find] = dict_spam_raw.get(find,1) #若是新的文字因為後面get找不到這個索引會返回逗點後面的值
                    dict_genuine_raw[find] = dict_genuine_raw.get(find,0)
        else:
            for find in finds:
                if len(find)<ignore: continue
                find = find.lower()
                try:
                    dict_genuine_raw[find] = dict_genuine_raw[find] + 1
                except:	
                    dict_genuine_raw[find] = dict_genuine_raw.get(find,1)
                    dict_spam_raw[find] = dict_spam_raw.get(find,0)

        word_set = set()
        for find in finds:
            if len(find)<ignore: continue
            find = find.lower()
            if not(find in word_set):
                try:
                    dict_IDF[find] = dict_IDF[find] + 1
                except:	
                    dict_IDF[find] = dict_IDF.get(find,1)
            word_set.add(find)
    word_df = pd.DataFrame(list(zip(dict_genuine_raw.keys(),dict_genuine_raw.values(),dict_spam_raw.values(),dict_IDF.values())))
    word_df.columns = ['keyword','genuine','spam','IDF']
    word_df['genuine'] = word_df['genuine'].astype('float')/data_rawtrain[data_rawtrain['label']=='genuine'].shape[0]
    word_df['spam'] = word_df['spam'].astype('float')/data_rawtrain[data_rawtrain['label']=='spam'].shape[0]
    word_df['IDF'] = np.log10(word_df.shape[0]/word_df['IDF'].astype('float'))
    word_df['genuine_IDF'] = word_df['genuine']*word_df['IDF']
    word_df['spam_IDF'] = word_df['spam']*word_df['IDF']
    word_df['diff']=word_df['spam_IDF']-word_df['genuine_IDF']
    selected_spam_key = word_df.sort_values('diff',ascending=False)  
    keyword_dict = dict()
    i = 0
    for word in selected_spam_key.head(size_table).keyword:
        keyword_dict.update({word.strip():i})
        i+=1
    return keyword_dict   
# build a tabu list based on the training data
size_table = 200                 # 多少特徵維度去分類SPAM
word_len_ignored = 3            # 忽略那些比這個還要小的字詞
keyword_dict=generate_key_list(data_rawtrain, size_table, word_len_ignored)


In [88]:
#排序越前面的代表出現在SPAM的機率越大
for key,value in keyword_dict.items():
    print('{key}:{value}'.format(key = key, value = value))

until:0
benefits:1
concern:2
fuelled:3
incident:4
shoranur:5
fated:6
belongs:7
programs:8
accumulation:9
predicting:10
amount:11
risks:12
amazing:13
deliver:14
relatives:15
plenty:16
residency:17
permanent:18
registration:19
begun:20
medicine:21
mmmmm:22
computers:23
prior:24
grief:25
violence:26
environment:27
behind:28
flute:29
dabbles:30
newscaster:31
rolled:32
dock:33
korli:34
achieve:35
tui:36
motive:37
tor:38
ultimately:39
superior:40
pressies:41
goods:42
someplace:43
advice:44
burnt:45
promptly:46
honestly:47
terrific:48
misplaced:49
ktv:50
splendid:51
released:52
necessity:53
ego:54
strange:55
struggling:56
drama:57
excused:58
greetings:59
papers:60
shoving:61
appointments:62
drizzling:63
srs:64
kanji:65
jungle:66
pansy:67
barcelona:68
easiest:69
jenne:70
key:71
locks:72
lock:73
reppurcussions:74
cosign:75
bognor:76
hcl:77
statements:78
pleasant:79
props:80
citylink:81
yunny:82
arrival:83
maga:84
whr:85
happens:86
afford:87
fwiw:88
reliant:89
bids:90
cars:91
resume:92
dai:93
in

# 4.將Train 和 Test資料轉換為特徵向量

In [112]:
def convert_Content(content, keyword_dict):
    #判斷是否有此特徵
    m = len(keyword_dict) #維度數量
    res = np.int_(np.zeros(m)) #建置一個幾維度的向量
    finds = re.findall('[A-Za-z]+', content) #將文本內容切割(類似中文斷詞)
    for find in finds:
        find=find.lower() #改小寫
        try:
            #若比對完有此特徵則特徵改為1
            i = keyword_dict[find] 
            res[i]=1
        except:
            continue
    return res
def raw2feature(data_rawtrain,data_rawtest,keyword_dict):
    n_train = data_rawtrain.shape[0]
    n_test = data_rawtest.shape[0]
    m = len(keyword_dict)
    X_train = np.zeros((n_train,m));
    X_test = np.zeros((n_test,m));
    Y_train = np.int_(data_rawtrain.label=='spam')
    Y_test = np.int_(data_rawtest.label=='spam')
    for i in range(n_train):
        X_train[i,:] = convert_Content(data_rawtrain.iloc[i].content, keyword_dict)
    for i in range(n_test):
        X_test[i,:] = convert_Content(data_rawtest.iloc[i].content, keyword_dict)
        
    return [X_train,Y_train],[X_test,Y_test]
     
Train,Test=raw2feature(data_rawtrain,data_rawtest,keyword_dict)


[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([0, 0, 1, ..., 0, 0, 0])]

# 5.建置分類器NB和RF

In [113]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB       
def learn(Train):
    model_NB = BernoulliNB()
    model_NB.fit(Train[0], Train[1])
    Y_hat_NB = model_NB.predict(Train[0])

    model_RF = RandomForestClassifier(n_estimators=10, max_depth=None,\
                                 min_samples_split=2, random_state=0)
    model_RF.fit(Train[0], Train[1])
    Y_hat_RF = model_RF.predict(Train[0])
    
    n=np.size(Train[1])
    print('Training Accuarcy NBclassifier : {:.2f}％'.format(sum(np.int_(Y_hat_NB==Train[1]))*100./n))
    print('Training Accuarcy RF: {:.2f}％'.format(sum(np.int_(Y_hat_RF==Train[1]))*100./n))
    return model_NB,model_RF
# train the Random Forest and the Naive Bayes Model using training data
model_NB,model_RF=learn(Train)

Training Accuarcy NBclassifier : 85.58％
Training Accuarcy RF: 86.54％


# 6.分類器NB和RF預測test set

In [114]:
def test(Test,model):
    Y_hat = model.predict(Test[0])
    n=np.size(Test[1])
    print ('Testing Accuarcy: {:.2f}％ ({})'.format(sum(np.int_(Y_hat==Test[1]))*100./n,model.__module__))
# Test Model using testing data
test(Test,model_NB)
test(Test,model_RF)
#######
def predictSMS(SMS,model,keyword_dict):
    X = convert_Content(SMS, keyword_dict)
    Y_hat = model.predict(X.reshape(1,-1))
    if int(Y_hat) == 1:
        print ('SPAM: {}'.format(SMS))
    else:
        print ('HAM: {}'.format(SMS))    

inputstr=('Call back for anytime.')
predictSMS(inputstr,model_NB,keyword_dict)

inputstr=('All goods will always be special until October 20')
predictSMS(inputstr,model_NB,keyword_dict)


Testing Accuarcy: 85.85％ (sklearn.naive_bayes)
Testing Accuarcy: 86.73％ (sklearn.ensemble.forest)
HAM: Call back for anytime.
SPAM: All goods will always be special until October 20
